<a href="https://colab.research.google.com/github/EdiciusRift/GEODASH/blob/main/GEODASH_Geotagging_and_Object_Detection_using_Dashcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GEODASH - Geotagging and Object Detection using Dashcam**

Sets up the configuration files of the YoloV4 Object Detection Model, and Monodepth2:

In [ ]:
!git clone https://github.com/nianticlabs/monodepth2.git
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'monodepth2'...
remote: Enumerating objects: 180, done.
remote: Total 180 (delta 0), reused 0 (delta 0), pack-reused 180
Receiving objects: 100% (180/180), 10.27 MiB | 26.42 MiB/s, done.
Resolving deltas: 100% (89/89), done.
Cloning into 'darknet'...
remote: Enumerating objects: 15424, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 15424 (delta 0), reused 1 (delta 0), pack-reused 15423
Receiving objects: 100% (15424/15424), 14.05 MiB | 20.97 MiB/s, done.
Resolving deltas: 100% (10364/10364), done.


In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!ln -s /content/gdrive/My\ Drive/ /mydrive

/
Mounted at /content/gdrive


In [ ]:
%cd /content/darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

/content/darknet


In [ ]:
# build darknet 
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old ==

In [ ]:
%cd data/
!find -maxdepth 1 -type f -exec rm -rf {} \;
%cd ..

%rm -rf cfg/
%mkdir cfg

/content/darknet/data
/content/darknet


In [ ]:
video = [video_file_name]
GPS = [GPS_file_name]
Magnetometer = [Magnetometer_file_name]

In [ ]:
#copy the datasets zip file to the root darknet folder
!cp /mydrive/yolov4/obj.names ./data
!cp /mydrive/yolov4/obj.data  ./data
!cp /mydrive/yolov4/train.txt ./data
!cp /mydrive/yolov4/test.txt ./data
!cp /mydrive/yolov4/yolov4-custom3.cfg ./cfg
!cp /mydrive/yolov4/obj.zip ../

# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../obj.zip -d data/

Streaming output truncated to the last 5000 lines.
  inflating: data/newTr/104.jpg      
  inflating: data/newTr/104.txt      
  inflating: data/newTr/104_Flipped.jpg  
  inflating: data/newTr/104_Flipped.txt  
  inflating: data/newTr/104_Flipped_HSV.jpg  
  inflating: data/newTr/104_Flipped_HSV.txt  
  inflating: data/newTr/104_HSV.jpg  
  inflating: data/newTr/104_HSV.txt  
  inflating: data/newTr/105.jpg      
  inflating: data/newTr/105.txt      
  inflating: data/newTr/105_Flipped.jpg  
  inflating: data/newTr/105_Flipped.txt  
  inflating: data/newTr/105_Flipped_HSV.jpg  
  inflating: data/newTr/105_Flipped_HSV.txt  
  inflating: data/newTr/105_HSV.jpg  
  inflating: data/newTr/105_HSV.txt  
  inflating: data/newTr/106.jpg      
  inflating: data/newTr/106.txt      
  inflating: data/newTr/106_Flipped.jpg  
  inflating: data/newTr/106_Flipped.txt  
  inflating: data/newTr/106_Flipped_HSV.jpg  
  inflating: data/newTr/106_Flipped_HSV.txt  
  inflating: data/newTr/106_HSV.jpg  
  i

In [ ]:
#set custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-custom3.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-custom3.cfg
%cd ..

/content/darknet/cfg
/content/darknet


In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))


Haversine Formula:

In [ ]:
from math import radians, cos, sin, asin, sqrt
import math

def haversine(lat1, lon1, lat2, lon2):

      R = 6372.8

      dLat = radians(lat2 - lat1)
      dLon = radians(lon2 - lon1)
      lat1 = radians(lat1)
      lat2 = radians(lat2)

      a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
      c = 2*asin(sqrt(a))

      return R * c

def bearing(lat1, lon1, lat2, lon2):
	rlat1 = math.radians(lat1)
	rlat2 = math.radians(lat2)
	rlon1 = math.radians(lon1)
	rlon2 = math.radians(lon2)
	dlon = math.radians(lon2-lon1)

	b = math.atan2(math.sin(dlon)*math.cos(rlat2),math.cos(rlat1)*math.sin(rlat2)-math.sin(rlat1)*math.cos(rlat2)*math.cos(dlon)) # bearing calc
	bd = math.degrees(b)
	br,bn = divmod(bd+360,360) # the bearing remainder and final bearing
	
	return bn

Read the GPS file of the video to find the frames when the vehicle travels every 10 meters using Haversine formula

In [ ]:
lines1 = []
interval = []
FPS = 30

with open("/content/gdrive/MyDrive/yolov4/2ndGPS_1.txt", 'r') as f:
  for line in f:
    lines1.append(line)

i = 0
j = 1

while j<len(lines1):
  lat1, lon1 = (lines1[i].split(' '))
  lat2, lon2 = lines1[j].split(' ')
  if(haversine(float(lon1), float(lat1), float(lon2), float(lat2)) >= 0.01):
    interval.append(j)
    i = j
    j += 1
  else:
    j+=1

f.close()

Save the frames when the vehicle travels every 10 meters and save the files names to a text file

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture("/content/gdrive/MyDrive/yolov4/sNov3_200.mp4")

if(cap.isOpened()== False):
  print("error")
i = 0
f = open("/content/detect.txt", 'w')
while(cap.isOpened()):
  ret, frame = cap.read()
  
  
  if ret == False:
    break
  if(i < len(interval)):
    if (cap.get(cv2.CAP_PROP_POS_FRAMES) == interval[i]*FPS+1):
      cv2.imwrite('/content/Capture_'+str(i+1)+'.jpg', frame)
      f.write('/content/Capture_'+str(i+1)+'.jpg\n')
      i+=1

f.close()
cap.release()
cv2.destroyAllWindows()



Use the custom Object Detection Model to the images stated in the text file

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-custom3.cfg /mydrive/yolov4/backup/yolov4-custom3_final.weights -dont_show -ext_output -out /content/result.json < /content/detect.txt -thresh 0.85

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv   

Get the category, bounding boxes, and confidence levels of the detected traffic signs

In [ ]:
import json

def get_data(distros_dict):
    json_to_variable = []
    # For every frame.
    for distro in distros_dict:
        filename = distro['filename']
        if len(distro['objects']) != 0:
            # For every detection.
            for obj in range(len(distro['objects'])):
                # Get values.
                frame_id = distro['frame_id']
                class_id = distro['objects'][obj]["class_id"]
                x = distro['objects'][obj]["relative_coordinates"]["center_x"]
                y = distro['objects'][obj]["relative_coordinates"]["center_y"]
                width = distro['objects'][obj]["relative_coordinates"]["width"]
                height = distro['objects'][obj]["relative_coordinates"]["height"]
                confidence = distro['objects'][obj]["confidence"]
                # And save them.
                print(f"{frame_id} {class_id} {x} {y} {width} {height} {confidence}")
                json_to_variable.append([frame_id, class_id, x, y, width, height, confidence])

    return json_to_variable

with open('/content/result.json', 'r') as f:
    distros_dict_train = json.load(f)
    
boundboxes = get_data(distros_dict_train)

1 0 0.585586 0.562319 0.021637 0.045171 0.992664
2 0 0.756971 0.494561 0.044526 0.076973 0.998383
3 1 0.586272 0.574587 0.020033 0.033736 0.904595
4 1 0.629755 0.536224 0.027724 0.05153 0.992205
5 1 0.83614 0.376262 0.086296 0.138236 0.989534
17 1 0.563086 0.574818 0.019472 0.032543 0.982802
18 1 0.621506 0.530482 0.02944 0.057442 0.996071
25 0 0.28507 0.584111 0.014348 0.021961 0.850434
27 0 0.07144 0.56648 0.027413 0.04552 0.975647
30 1 0.267018 0.579387 0.017907 0.026448 0.926497
31 1 0.188511 0.55904 0.022127 0.0393 0.97614
32 1 0.027775 0.524229 0.033142 0.054553 0.915114
34 1 0.536549 0.585896 0.022202 0.025244 0.951103
37 1 0.776973 0.438926 0.06951 0.098151 0.998641
41 0 0.166561 0.559252 0.022822 0.039046 0.990953
42 1 0.682834 0.583447 0.030518 0.0655 0.9493
48 1 0.931122 0.427749 0.078833 0.114109 0.991324
55 0 0.128765 0.547319 0.022394 0.042063 0.916119


In [ ]:
def yolobbox2bbox(x,y,w,h):
    x1, y1 = x-w/2, y-h/2
    x2, y2 = x+w/2, y+h/2
    return x1, y1, x2, y2

In [ ]:
indexes = []
convbbox = []
f = open('/coords.txt', 'w')
for i in range(len(boundboxes)):
  frame = boundboxes[i][0]
  indexes.append(frame)
  classid = boundboxes[i][1]
  ox = boundboxes[i][2]
  oy = boundboxes[i][3]
  ow = boundboxes[i][4]
  oh = boundboxes[i][5]
  conf = boundboxes[i][6]
  x1, y1, x2, y2 = yolobbox2bbox(ox, oy, ow, oh)
  f.write(str(i) + ' ' + str(x1) + ' ' + str(y1) + ' ' + str(x2) + ' ' + str(y2) +'\n')
f.close()
bboxindexes = [indexes.index(x) for x in set(indexes)]

In [ ]:
from __future__ import absolute_import, division, print_function
%matplotlib inline

import os
import numpy as np
import PIL.Image as pil
import matplotlib.pyplot as plt
import cv2
import torch
from torchvision import transforms

import sys
sys.path.append('/content/monodepth2')

import networks
from utils import download_model_if_doesnt_exist
from layers import disp_to_depth
from evaluate_depth import STEREO_SCALE_FACTOR

model_name = "mono+stereo_1024x320"

download_model_if_doesnt_exist(model_name)
encoder_path = os.path.join("models", model_name, "encoder.pth")
depth_decoder_path = os.path.join("models", model_name, "depth.pth")

# LOADING PRETRAINED MODEL
encoder = networks.ResnetEncoder(18, False)
depth_decoder = networks.DepthDecoder(num_ch_enc=encoder.num_ch_enc, scales=range(4))

loaded_dict_enc = torch.load(encoder_path, map_location='cpu')
filtered_dict_enc = {k: v for k, v in loaded_dict_enc.items() if k in encoder.state_dict()}
encoder.load_state_dict(filtered_dict_enc)

loaded_dict = torch.load(depth_decoder_path, map_location='cpu')
depth_decoder.load_state_dict(loaded_dict)

encoder.eval()
depth_decoder.eval();


for i in range(len(bboxindexes)):  
  print(i+1)
  image_path = "/content/Capture_"+str(indexes[bboxindexes[i]])+".jpg"

  paths = [image_path]
  output_directory = os.path.dirname(image_path)

  input_image = pil.open(image_path).convert('RGB')
  original_width, original_height = input_image.size

  feed_height = loaded_dict_enc['height']
  feed_width = loaded_dict_enc['width']
  input_image_resized = input_image.resize((feed_width, feed_height), pil.LANCZOS)

  input_image_pytorch = transforms.ToTensor()(input_image_resized).unsqueeze(0)

  with torch.no_grad():
      features = encoder(input_image_pytorch)
      outputs = depth_decoder(features)

  disp = outputs[("disp", 0)]

  output_name = os.path.splitext(os.path.basename(image_path))[0]
  scaled_disp, depth = disp_to_depth(disp, 0.1, 100)
  name_dest_npy = os.path.join("/{}depth.npy".format(output_name))
  metric_depth = STEREO_SCALE_FACTOR * depth.cpu().numpy()
  np.save(name_dest_npy, metric_depth)

  disp_resized = torch.nn.functional.interpolate(disp,
      (original_height, original_width), mode="bilinear", align_corners=False)

  disp_resized_np = disp_resized.squeeze().cpu().numpy()
  vmax = np.percentile(disp_resized_np, 95)
  plt.imsave('/content/Capture'+str(indexes[bboxindexes[i]])+'_output.jpg', disp_resized_np, cmap='magma', vmax=vmax)

-> Downloading pretrained model to models/mono+stereo_1024x320.zip
   Unzipping model...
   Model unzipped to models/mono+stereo_1024x320
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


Find distance of the detected object from the camera

In [ ]:
pxcoord = []
with open('/content/coords.txt', 'r') as f:
  for lines in f:
    pxcoord.append(lines.strip())

In [ ]:
import math

angles = []
adjangles = []

for i in range(len(indexes)):
  x1, y1 = pxcoord[i].split(' ')
  angle = 180 - math.degrees(math.atan2(int(float(y1)) - 1080, int(float(x1)) - 960)) %  180

  angles.append(angle)

  if angle > 90:
    angle = 90 - angle

  adjangles.append(angle)


In [ ]:
distance = []
for i in range(len(indexes)):
  x1, y1 = pxcoord[i].split(' ')
  data = np.load('/Capture'+ str(indexes[i]) + '_depth.npy')
  distance.append(data[0,0,int(float(y1)/320), int(float(x1)/1024)] / math.sin(math.radians(angles[i])))

Get the orientation of the vehicle by comparing the magnetometer data to the Haversine formula data

In [ ]:
magdata = []
gpsdata = []
orientation = []
with open('/content/gdrive/MyDrive/yolov4/2ndMAG.txt', 'r') as f:
  for lines in f:
    magdata.append(lines.strip())

f.close()

with open('/content/gdrive/MyDrive/yolov4/2ndGPS.txt', 'r') as f:
  for lines in f:
    gpsdata.append(lines.strip())
f.close()

for i in range(len(indexes)):
  if interval[indexes[i]-1] == 0:
    lon1, lat1 = gpsdata[interval[indexes[i]-1]].split()
    lon2, lat2 = gpsdata[interval[indexes[i]]].split()

    brng = bearing(float(lon1), float(lat1), float(lon2), float(lat2))
      
    orientation.append(magdata[interval[indexes[i]-1]])

  else:
    lon1, lat1 = gpsdata[interval[indexes[i]-2]].split()
    lon2, lat2 = gpsdata[interval[indexes[i]-1]].split()

    brng = bearing(float(lon1), float(lat1), float(lon2), float(lat2))

    orientation.append(magdata[interval[indexes[i]]])


Get angles of the detected traffic signs relative to the FOV of the camera

In [ ]:
fovangles = []
for i in range(len(indexes)):
  phangle = adjangles[i] / 90
  camangle = 41 * phangle
  fovangles.append(camangle)

Add the FOV angle to the orientation of the vehicle to get the effective angle of the traffic sign from the vehicle

In [ ]:
effangle = []

for i in range(len(indexes)):
  effangle.append(float(fovangles[i] + float(orientation[i])))


Convert the estimated distance to decimal degrees to add to the current coordinates of the vehicle

In [ ]:
lat_o = []
lon_o = []
curr_coord = []


for i in range(len(indexes)):
  lat_o.append(float(distance[i]) * math.cos(math.radians(float(effangle[i]))) / 1.11 * 0.00001)
  lon_o.append(float(distance[i]) * math.sin(math.radians(float(effangle[i]))) / 1.11 * 0.00001)
  lon1, lat1 = gpsdata[interval[indexes[i]]].split()
  newlat = lat_o[i] + float(lat1)
  newlon = lon_o[i] + float(lon1)
  comb = newlat, newlon
  curr_coord.append(comb)




Create the database

In [ ]:
!apt install mongodb
!service mongodb start

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb-clients
  mongodb-server mongodb-server-core
The following NEW packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb mongodb-clients
  mongodb-server mongodb-server-core
0 upgraded, 8 newly installed, 0 to remove and 49 not upgraded.
Need to get 53.1 MB of archives.
After this operation, 215 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpcap0.8 amd64 1.8.1-6ubuntu1.18.04.2 [118 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libstemmer0d amd64 0+svn585-1build1 [62.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 lib

In [ ]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['mydb']

collections = db.list_collection_names()
for coll in collections:
  col = db[coll]
  col.drop()

for coll in collections:
   print(coll)

coll = db['traffic_signs']

Without averaging

In [ ]:
id = coll.count_documents({}) + 1
for i in range(len(indexes)):
  lat1 = curr_coord[i][0]
  lon1 = curr_coord[i][1]
  classid = int(boundboxes[i][1])
  
  if classid == 0:
      cat = "regulatory"
  elif classid == 1:
    cat = "warning"
  else:
    cat = "guide"
    
  data = {'_id': id, 'latitude': lat1, 'longitude': lon1, 'category': cat}
  coll.insert_one(data)
  id+=1

In [ ]:
!pip install gmplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 164 kB 5.3 MB/s 


In [ ]:
import gmplot
from google.colab import files

latitude = []
longitude = []
categ = []

results = coll.find()
for i in results:
  latitude.append(i['latitude'])
  longitude.append(i['longitude'])
  categ.append(i['category'])

latList1 = latitude[int((len(latitude)/2)):]
longList1 = longitude[int((len(longitude)/2)):]
gmap = gmplot.GoogleMapPlotter(latList1[0], longList1[0], 20)

for i in range(len(latitude)):
  if categ[i] == "warning":
    color = "yellow"
  elif categ[i] == "regulatory":
    color = "red"
  else:
    color = "blue"
  gmap.marker(latitude[i], longitude[i], precision = 10, color=color, title=categ[i])

gmap.draw('/content/output.html')
files.download('/content/output.html') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

With averaging

In [ ]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['mydb']

collections = db.list_collection_names()
for coll in collections:
  col = db[coll]
  col.drop()

for coll in collections:
   print(coll)

coll = db['traffic_signs']

traffic_signs


In [ ]:
id = coll.count_documents({}) + 1
for i in range(len(indexes)):
  lat1 = curr_coord[i][0]
  lon1 = curr_coord[i][1]
  classid = int(boundboxes[i][1])
  
  if classid == 0:
      cat = "regulatory"
  elif classid == 1:
    cat = "warning"
  else:
    cat = "guide"

  if id != 1:
    for j in range(id):
      value = coll.find_one({"_id": j+1})
      if value:
        lat2 = value['latitude']
        lon2 = value['longitude']
        cat2 = value['category']
        weight = value['weight']
      if cat == cat2:
        if(haversine(lat1, lon1, lat2, lon2) <= 0.005):
          newlat = (lat1 * weight + lat2 * (1/math.sqrt(abs(distance[i])))) / (weight + 1/math.sqrt(abs(distance[i])))
          newlon = (lon1 * weight + lon2 * (1/math.sqrt(abs(distance[i])))) / (weight + 1/math.sqrt(abs(distance[i])))
          weight = (weight + 1/math.sqrt(abs(distance[i])))
          coll.update_one({"_id":id-1},{"$set":{"latitude":newlat, "longitude":newlon, "weight":weight}})
          res = coll.find_one({"_id":id})
          break
        else:
          if j == id - 1:
            weight = 1/math.sqrt(abs(distance[i]))
            data = {'_id': id, 'latitude': lat1, 'longitude': lon1, 'category': cat, "weight":weight}
            coll.insert_one(data)
            id+=1
      else:
        if j == id - 1:
          weight = 1/math.sqrt(abs(distance[i]))
          data = {'_id': id, 'latitude': lat1, 'longitude': lon1, 'category': cat, "weight":weight}
          coll.insert_one(data)
          id+=1

  else:
    weight = 1/math.sqrt(abs(distance[i]))
    data = {'_id': id, 'latitude': lat1, 'longitude': lon1, 'category': cat, "weight":weight}
    coll.insert_one(data)
    id+=1

Output the records of the database to Google Maps

In [ ]:
import gmplot
from google.colab import files

latitude = []
longitude = []
categ = []

results = coll.find()
for i in results:
  latitude.append(i['latitude'])
  longitude.append(i['longitude'])
  categ.append(i['category'])

latList1 = latitude[int((len(latitude)/2)):]
longList1 = longitude[int((len(longitude)/2)):]
gmap = gmplot.GoogleMapPlotter(latList1[0], longList1[0], 20)

for i in range(len(latitude)):
  if categ[i] == "warning":
    color = "yellow"
  elif categ[i] == "regulatory":
    color = "red"
  else:
    color = "blue"
  gmap.marker(latitude[i], longitude[i], precision = 10, color=color, title=categ[i])

gmap.draw('/content/output.html')
files.download('/content/output.html') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



---


For testing the mAP @ 0.5 IoU of the custom Object Detection Model:



In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-custom3.cfg /mydrive/yolov4/backup/yolov4-custom3_last.weights -points 0 --iou 0.75

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv  